### Extract field from invoices using GPT-4 Vision

In [1]:
#Install Dependencies
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import packages
from dotenv import load_dotenv
import os
import glob
import itertools
import base64
from mimetypes import guess_type
import json
import pandas as pd
import requests
import time
from datetime import datetime


#### Get config details

In [3]:
# Load environment variables
load_dotenv('../config.env')

# Get the connection strings and other secrets
api_base = os.getenv('AOAI_ENDPOINT')
api_key = os.getenv('AOAI_KEY')
deployment_name = os.getenv('AOAI_DEPLOYMENT_NAME')
api_version= os.getenv('AOAI_API_VERSION')
vision_key =  os.getenv('AZURE_VISION_API_KEY')
vision_endpoint =  os.getenv('AZURE_VISION_API_ENDPOINT')
api_url=f"{api_base}openai/deployments/{deployment_name}/extensions/chat/completions?api-version={api_version}"
img_path = os.getenv('IMAGE_PATH')

# Print to verify
print(f'Azure OpenAI Url: {api_url}')
print(f'Azure OpenAI Deployment Name: {deployment_name}')
print(f'Azure OpenAI API Version: {api_version}')
print(f'Azure Vision Url: {vision_endpoint}')
print(f'Image path: {img_path}')


Azure OpenAI Url: https://oai-slinvoice-azureai-001.openai.azure.com/openai/deployments/gpt-4-vision/extensions/chat/completions?api-version=2023-12-01-preview
Azure OpenAI Deployment Name: gpt-4-vision
Azure OpenAI API Version: 2023-12-01-preview
Azure Vision Url: https://cv-invoicecv-azureai-001.cognitiveservices.azure.com/
Image path: invoice/png/15-05-2024/


#### Helper functions

In [4]:
# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Function to create or update dataframe
def create_or_update_dataframe(data):
    global df
    df1 = pd.json_normalize(data)

    # If df is empty, create a new DataFrame. Otherwise, append to the existing DataFrame
    if df.empty:
        df = df1
    else:
        merged_df = pd.concat([df, df1], ignore_index=True, sort=False)
        df = merged_df
    df.head()
    return df



In [5]:
# Process Invoice with rest call
def process_invoice(files, prompt):
    # Base messages list
    messages = [
        { "role": "system", "content": prompt },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Please extarct relevant fileds from the invoice"
            }
        ] } 
    ]

    #For each file get the image url and add image_url section to the base message
    for file in files:
        # Get image data url
        image = local_image_to_data_url(f'{img_path}{file}')
        image_url_section = { 
        "type": "image_url",
        "image_url": {
            "url": image
            }
        }
        # Add image url to the message
        messages[1]['content'].append(image_url_section)



    # Define the data sources
    data_sources = [
        {
            "type": "AzureComputerVision",
            "parameters": {
                "endpoint": vision_endpoint,
                "key": vision_key
            }
        }
    ]

    # Define the enhancements
    enhancements = {
        "ocr": {
            "enabled": True
        }
    }

    payload = json.dumps({
        "enhancements": enhancements,
        "dataSources":data_sources,
        "messages": messages,
        "max_tokens": 4000,
        "temperature": 0
    })
    headers = {
    'api-key': api_key,
    'Content-Type': 'application/json'
    }

    while True:
        response = requests.request("POST", api_url, headers=headers, data=payload)
        if response.status_code != 429:
            break
        
        # Sleep for 5 seconds before retrying in case of 429 error.
        time.sleep(5)   
    
    # Convert the response to a dictionary
    response_dict = json.loads(response.text)


    # Extract the 'choices' field which contains the model's responses
    choices = response_dict.get('choices')

    # Get the text of the first response
    response_text = choices[0].get('message').get('content') if choices else None
    
    response_text=response_text.replace("```json\n", "").replace("\n```", "")

    # Load the JSON string into a Python dictionary
    invoice_details = json.loads(response_text)

    return invoice_details

#### Set up prompt

In [6]:
# Define System Prompt
prompt = ''' 
You are a helpul assistant who understands various invoice formats.
Given an invoice, which could be a single image or multiple images, your task is to extract specific fields from the invoice images. 
The fields to be extracted are as follows:

    Vendor Name
    Vendor VAT Registration Number
    Invoice or Credit
    Invoice Number
    Credit Reference
    Invoice Date
    Currency
    Net Amount
    Tax Amount
    Freight
    Misc Charges
    Total Amount
    Purchase Order Number

###Guidelines:###
	If an invoice contains multiple Purchase Orders (POs), the Purchase Order field will repeat. This field **MUST BE** kept as an array. 
	Purchase Order field might not be present in all invoices. You **MUST** return an empty array in such cases and **MUST NOT** fabricate values. 
	Purchase Order field usually starts with "PU" and it's 10 characters long including "PU" in the beginning. Please ignore anything after 10 characters or before "PU". eg: If PO number is XX-PU11111111-01, you must return PU11111111.


	The Vendor Name and Vendor VAT Registration Number are associated with the entity issuing the invoice, not with the 'ship to', 'sold to', 'customer', or 'billed to' fields. The Vendor Name is not 'RENISHAW', 'RENISHAW PLC' or anything similar.
	You **MUST** return the Vendor/Supplier VAT Registration Number (VAT ID) as specified in the invoice image.

	The Invoice Date **MUST** always be returned in the 'dd/MM/yyyy' format. If the Invoice date is in a different format, eg: yy MM dd, convert it to the 'dd/MM/yyyy' format and you **MUST NOT** fabricate this value.
	
	The Currency field **MUST** always be returned as an ISO 4217 code.

	You **MUST** refrain from fabricating any text or numerical value that is not present in the image file.
	You **MUST NOT** calculate the Net Amount, Tax Amount, Freight, Misc Charges, and Total Amount. These values **MUST BE** based on the text from the image. The Total Amount is always the total amount due. Remove any currency symbols (eg: £, $, GBP, USD etc) from these fields and only return the number parts.
    Please be aware that Freight might be present as a line item in some invoices. You ** MUST NOT** add Freight as Misc Charges.

	If a field is not present, You **MUST** return the field name without any value.You **MUST NOT** fabricate any values.

	The result should be returned in the following JSON format and **MUST NOT** include any other text:
		{
		"VendorName": "",
		"VendorVATRegistrationNumber": "",
		"InvoiceOrCredit": "",
		"InvoiceNumber": "",
		"CreditReference": "",
		"InvoiceDate": "",
		"Currency": "",
		"NetAmount": "",
		"TaxAmount": "",
		"Freight": "",
		"MiscCharges": "",
		"TotalAmount": "",
		"Orders": [
			{
				"PONumber": ""
			}
		]
	}
###
'''

#### Process files in group & invoke openAI

In [8]:
# Global variable for the DataFrame
df = pd.DataFrame()

# Get all png files in the 'invoice/png' directory
files = glob.glob(f'{img_path}*.png')

# Remove 'invoice/png/' from all file names
files = [file.replace(img_path, '') for file in files]

# Group files based on the name part (after '_')
groups = itertools.groupby(sorted(files, key=lambda x: x.split('_')[1]), key=lambda x: x.split('_')[1])

for name, group in groups:
    # Sort files in each group based on the iteration part (before '_')
    sorted_group = sorted(list(group), key=lambda x: int(x.split('_')[0]))
    # Call your processing function for each group of files
    response=process_invoice(sorted_group, prompt)
    # Add file to the dictionry
    response['FileUrl'] = name
    print(response)
    print("\n")
    df = create_or_update_dataframe(response)



{'VendorName': 'Shenzhen Indus-connector Limited', 'VendorVATRegistrationNumber': '', 'InvoiceOrCredit': 'Invoice', 'InvoiceNumber': 'IC210130-01', 'CreditReference': '', 'InvoiceDate': '25/02/2021', 'Currency': 'USD', 'NetAmount': '2850.00', 'TaxAmount': '', 'Freight': '0.00', 'MiscCharges': '', 'TotalAmount': '2850.00', 'Orders': [], 'FileUrl': '681409.png'}


{'VendorName': 'Farnell', 'VendorVATRegistrationNumber': 'GB 169 6803 22', 'InvoiceOrCredit': 'Invoice', 'InvoiceNumber': '5792718', 'CreditReference': '', 'InvoiceDate': '20/10/2021', 'Currency': 'GBP', 'NetAmount': '14856.97', 'TaxAmount': '2971.39', 'Freight': '', 'MiscCharges': '', 'TotalAmount': '17828.36', 'Orders': [{'PONumber': 'PU03579185'}], 'FileUrl': '714365.png'}


{'VendorName': 'SWEETNAM & BRADLEY LTD', 'VendorVATRegistrationNumber': '930032965', 'InvoiceOrCredit': 'Invoice', 'InvoiceNumber': '74840', 'CreditReference': '', 'InvoiceDate': '17/12/2021', 'Currency': 'GBP', 'NetAmount': '1575.04', 'TaxAmount': '315.

In [9]:
df.head(25)

,VendorName,VendorVATRegistrationNumber,InvoiceOrCredit,InvoiceNumber,CreditReference,InvoiceDate,Currency,NetAmount,TaxAmount,Freight,MiscCharges,TotalAmount,Orders,FileUrl
0,Shenzhen Indus-connector Limited,,Invoice,IC210130-01,,25/02/2021,USD,2850.00,,0.00,,2850.00,[],681409.png
1,Farnell,GB 169 6803 22,Invoice,5792718,,20/10/2021,GBP,14856.97,2971.39,,,17828.36,[{'PONumber': 'PU03579185'}],714365.png
2,SWEETNAM & BRADLEY LTD,930032965,Invoice,74840,,17/12/2021,GBP,1575.04,315.01,0.00,,1890.05,[{'PONumber': 'PU03581000'}],724393.png
3,TR FASTENINGS LIMITED,GB390068451,Invoice,621185,,17/12/2021,GBP,879.34,175.87,,,1055.21,[{'PONumber': 'PU03585370'}],724454.png
4,ERIKS Industrial Services Ltd.,GB 277 2632 40,Invoice,01808896,,23/12/2021,GBP,2980.00,596.00,,,3576.00,[{'PONumber': 'PU03576899'}],726035.png
5,Merlin Circuit Technology Ltd,GB 587 8909 59,Invoice,IN00096583,,29/11/2021,GBP,668.52,133.70,,,802.22,[{'PONumber': 'PU03581245'}],729221.png
6,SUNTAK TECHNOLOGY LIMITED,,Invoice,11252731,,19/01/2022,USD,26829.29,,,,26829.29,"[{'PONumber': 'PU03573159'}, {'PONumber': 'PU0...",731350.png
7,Avnet EMG Ltd,GB 382260166,Invoice,3302872260,,08/02/2022,GBP,36675.00,7335.00,,,44100.00,[{'PONumber': 'PU03591578'}],733178.png
8,"JINAN XINLEI PRECISION MACHINERY CO., LTD.",,Invoice,CH-220101,,05/01/2022,GBP,78425.00,,,,78425.00,[{'PONumber': 'PU03576307'}],735028.png
9,"MFS TECHNOLOGY (Hunan) CO., LTD",,Invoice,HMFS76344,,19/02/2022,USD,,0,,,38930.00,"[{'PONumber': 'PU03572540'}, {'PONumber': 'PU0...",735033.png


In [10]:
#Save results
# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')

# Define the directory path
dir_path = os.path.join('analyzed', timestamp)

# Create the directory if it doesn't exist
os.makedirs(dir_path, exist_ok=True)

# Define the file path
file_path = os.path.join(dir_path, 'result.csv')
# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

In [12]:
df1 = df.explode('Orders')

# Check if 'Orders' is a dictionary before trying to access 'PONumber'
df1['Orders'] = df1['Orders'].apply(lambda x: x['PONumber'] if isinstance(x, dict) else None)
df1[['Orders', 'FileUrl']].head(100)


,Orders,FileUrl
0,None,681409.png
1,PU03579185,714365.png
2,PU03581000,724393.png
3,PU03585370,724454.png
4,PU03576899,726035.png
5,PU03581245,729221.png
6,PU03573159,731350.png
6,PU03573229,731350.png
6,PU03574924,731350.png
6,PU03574919,731350.png
